In [ ]:
def plot(samples, weights, lims=False, logscale = False):   
    if logscale is True:
        for c in samples.columns:
            samples[c] = samples[c].apply(lambda x: np.log(x))
            
    kde = scipy.stats.gaussian_kde(samples.T.values.tolist(),weights=weights)
    size = 50000
    samples = kde.resample(size).T
    samples = pd.DataFrame(data=samples,columns=['mutation rate', 'trisomy rate', '2n+1 fitness', '2n+1* fitness', '2n* fitness'])
    weights = np.full(size,1/size)
    params_hat = scipy.optimize.minimize(lambda x: -kde.logpdf(x) ,samples.median().values)['x']
#     params_hat = samples.median()
    print(params_hat)
    mutl = (10.0**-9,10.0**-6)
    tril = (10.0**-5,10.0**-2)
    limits = {
        'mutation rate':mutl,
        'trisomy rate':tril,
        '2n+1 fitness':(1,1.1),
        '2n+1* fitness':(1,1.1),
        '2n* fitness':(1,1.1)}
    
    if logscale is True:
        limits = {i:(np.log(v[0]),np.log(v[1])) for (i,v) in limits.items()}
        
    if lims is True:
        axes = plot_kde_matrix_for_paper(
            samples, weights,numx=1000,limits=limits,numy=1000,refval={c:p for c,p in  zip(samples.columns,params_hat)}
        )
    else:
        axes = plot_kde_matrix_for_paper(
            samples, weights,numx=1000,numy=1000,refval={c:p for c,p in  zip(samples.columns,params_hat)}
        )